<a href="https://colab.research.google.com/github/teacherSsamko/DL-study/blob/main/w5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [5주차] 심화과제: 다양한 형태의 입력을 가지는 LLM 서비스 개발

## 코드 리뷰 LLM 서비스


코드 리뷰 LLM 서비스는 다음과 같이 구성됩니다:

- **입력:**  코드 리뷰 서비스의 입력은 다음 두 가지 중 하나로 결정하시면 됩니다:
    - **하나의 함수와 같이 짧은 코드:** 사용자에게 직접 코드 리뷰를 받고 싶은 코드를 입력으로 받습니다. 함수 구현과 같이 짧은 코드가 될 것 입니다.
    - **Github repository:** 특정 github repository 링크를 입력받아, 프로젝트의 모든 코드들을 리뷰 받을 수도 있습니다.
- **방법론:** 입력 형태에 따라 다음 두 가지 방법 중 하나를 선택하시면 됩니다:
    - **짧은 코드:** 만약 짧은 길이의 코드를 입력받는 경우, 간단한 prompting을 통해 LLM만으로 코드 리뷰를 받으시면 됩니다.
    - **Github repository:** repository를 입력으로 받는 경우, 모든 코드들을 concat하여 짧은 코드와 같이 prompting 할 수도 있습니다. 아니면 RAG를 통해 문제 있는 부분들만 추출한 후(이 경우 prompt를 어떻게 정할지가 중요합니다), 이들을 가지고 코드 리뷰를 진행할 수도 있습니다.
- **출력:** 주어진 코드에서 문제되는 부분과 왜 문제가 되는지에 대한 간단한 설명이 출력되어야 합니다.

위의 LLM 서비스를 다음 요구사항을 충족시키며 개발하시면 됩니다:

- 입력 및 방법론 결정
    - 위에서 주어진 선택지들 중에서 어떤 입력을 받으실지 정하셔야 합니다.
- 코드 리뷰 prompting 제안
    - 코드 리뷰가 출력되도록 prompt를 구성하셔야 합니다. 만약 RAG도 활용하셔야 하는 경우, 문제 있는 부분들을 우선 추출할 수 있도록 prompt를 설계하는 것이 중요할 것입니다.
- 실제 코드 리뷰 결과 출력

## 입력 및 방법론

github repository 링크를 입력 받아(입력), RAG를 통해(방법론) 코드 리뷰 진행

In [1]:
!pip install langchain-community langchain-chroma langchain-openai PyGithub sentence-transformers langchain-huggingface transformers

In [38]:
!pip install transformers -U

In [37]:
!pip uninstall chromadb

Found existing installation: chromadb 0.5.23
Uninstalling chromadb-0.5.23:
  Would remove:
    /usr/local/bin/chroma
    /usr/local/lib/python3.10/dist-packages/chromadb-0.5.23.dist-info/*
    /usr/local/lib/python3.10/dist-packages/chromadb/*
Proceed (Y/n)? Y
  Successfully uninstalled chromadb-0.5.23


In [2]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

## [MY CODE] 환경 변수 설정

### OpenAI

ChatOpenAI는 OPENAI_API_KEY를 환경변수로만 받고 있어서, 환경변수에 지정

관련 문서: [ChatOpenAI](https://python.langchain.com/docs/integrations/chat/openai/)

### LANGCHAIN

LANGCHAIN과 LANGSMITH 관련 환경변수 설정

관련 문서: [LANGSMITH](https://docs.smith.langchain.com/), [LANGCHAIN](https://smith.langchain.com/o/464e449d-6cf3-4e7a-a47f-5b96b912650d?paginationState=%7B%22pageIndex%22:0,%22pageSize%22:5%7D)

In [3]:
import os

from google.colab import userdata
import openai



# openai API KEY
openai_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

# langsmith
langsmith_key = userdata.get('LANGSMITH_KEY')
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = langsmith_key
os.environ["LANGSMITH_API_KEY"] = langsmith_key
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "sparta_w5_1"


llm = ChatOpenAI(model="gpt-4o-mini")

## [MY CODE] Github repository 가져오기

In [4]:
from github import Github
from github import Auth

# using an access token
git_access_token = userdata.get('GIT_TOKEN')
auth = Auth.Token(git_access_token)

# Public Web Github
g = Github(auth=auth)

# Repository 가져오기
repo_url = "https://github.com/teacherSsamko/lion_django_app"
repo_name = "/".join(repo_url.split("/")[-2:]) # "PyGithub/PyGithub"
print(repo_name)
repo = g.get_repo(repo_name)

# 리포지토리의 모든 파일 가져오기
contents = repo.get_contents("")
files = []

while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    else:
        files.append(file_content)

# 각 파일 내용 읽기
code_files = {file.path: file.decoded_content.decode() for file in files if file.path.endswith(('.py', '.js', '.ts', '.java', '.c', '.cpp'))}

teacherSsamko/lion_django_app


## [MY CODE] 코드 내용을 RAG용 벡터로 변환

RAG는 코드를 세그먼트로 나누고 각 세그먼트를 벡터화하여 저장합니다. 이를 위해 OpenAI의 text-embedding-ada-002 모델을 사용할 수 있습니다.


1.	코드 세그먼트화

  파일 내용을 일정 길이로 나누어 벡터화합니다.
2.	벡터 저장

  벡터를 Chroma에 저장합니다.
  > 추후 production 단계에서는 pinecone으로 업그레이드

In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# Chroma 데이터베이스 설정 및 초기화
embeddings = OpenAIEmbeddings()
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings,
    collection_name="code_segments"
)

# 데이터 업로드
def add_data_to_chroma(segments, metadata):
    # LangChain의 Document 형식으로 변환
    documents = [
        Document(
            page_content=segment,
            metadata=meta
        ) for segment, meta in zip(segments, metadata)
    ]

    # vectorstore에 documents 추가
    vectorstore.add_documents(documents)

for file_path, code in code_files.items():
    # Split the code into segments (you might want to use a more sophisticated approach)
    segments = code.split('\n')

    # Create metadata for each segment
    metadata = [{'file_path': file_path, 'line_number': i} for i in range(len(segments))]

    # Add the segments and metadata to the vectorstore
    add_data_to_chroma(segments, metadata)

## [MY CODE] RAG 기반 검색

In [26]:
# 쿼리를 통해 관련 벡터 검색
def search_relevant_segments(query: str, top_k: int = 5):
    # similarity_search 메서드를 사용하여 관련 문서 검색
    results = vectorstore.similarity_search(
        query=query,
        k=top_k
    )
    # 검색된 문서의 내용 반환
    return [doc.page_content for doc in results]

# 결과 출력
query = "잠재적인 버그, 스타일 문제, 성능 문제, 보안 취약점 등 특정 코드 블록이나 함수에서 개선할 수 있는 부분"
relevant_segments = search_relevant_segments(query)
relevant_segments

[')', ']', ']', ']', ']']

## [MY CODE] OpenAI로 코드 리뷰 생성

검색된 세그먼트를 기반으로 GPT 모델을 통해 리뷰를 생성합니다.

In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def generate_code_review(relevant_segments):
    # 프롬프트 템플릿 생성
    prompt = ChatPromptTemplate.from_template("""
    You are a code reviewer. Here are some code segments:
    {segments}

    Please provide:
    1. What was done well.
    2. What can be improved.
    """)

    # LLM 모델 초기화
    llm = ChatOpenAI(model="gpt-4")

    # 체인 실행
    chain = prompt | llm

    # 리뷰 생성
    response = chain.invoke({
        "segments": "\n".join(relevant_segments)
    })

    return response.content

# 리뷰 생성 및 출력
review = generate_code_review(relevant_segments)
print(review)

This code segment provided is unclear and seems incomplete, and multiple closing brackets are seemingly not matched with opening brackets. Thus, it's challenging to provide detailed feedback. However, based on the current context, here are some simple comments:

1. What was done well:

It is difficult to pinpoint what was done well as the code provided is incomplete and without context. 

2. What can be improved:

a) It's clear that the code is not syntactically valid, as there are closing brackets without corresponding opening ones. First priority is to fix the code syntax.

b) Variables, Objects, or Data which are supposed to be inside the braces are missing. This needs to be corrected.

c) If this broken code implies an arrangement of web addresses, then consider storing them in a data structure such as a list or a dictionary (if a mapping is required).

d) Leaving commented out code can make it less readable or confusing. Evaluate if the commented URL is necessary or else remove or

## [LOG] Code가 제대로 입력되지 않은것으로 보임

In [17]:
relevant_segments

[')',
 '#     "http://lion-lb-staging-18975820-e3aa639c372a.kr.lb.naverncp.com",',
 ']',
 ']',
 ']']

## [LOG] segment가 이상하게 출력되고 있음


In [19]:
code_files

{'lion_app/manage.py': '#!/usr/bin/env python\n"""Django\'s command-line utility for administrative tasks."""\nimport os\nimport sys\n\n\ndef main():\n    """Run administrative tasks."""\n    os.environ.setdefault("DJANGO_SETTINGS_MODULE", "lion_app.settings.local")\n    try:\n        from django.core.management import execute_from_command_line\n    except ImportError as exc:\n        raise ImportError(\n            "Couldn\'t import Django. Are you sure it\'s installed and "\n            "available on your PYTHONPATH environment variable? Did you "\n            "forget to activate a virtual environment?"\n        ) from exc\n    execute_from_command_line(sys.argv)\n\n\nif __name__ == "__main__":\n    main()\n',
 'monitoring/api_test.py': 'from abc import ABC, abstractmethod\nimport time\n\nimport requests\nfrom faker import Faker\n\n\nclass APIHandler(ABC):\n    def __init__(self, model: str, host: str = "http://localhost:8000"):\n        self.root_url = "/forum/topic"\n        self.m

In [24]:
# Assuming you want to retrieve all documents:
results = vectorstore.get(include=['metadatas'])

# Get the number of documents by counting the metadata entries
num_documents = len(results['metadatas'])

print(f"Number of documents in vectorstore: {num_documents}")

Number of documents in vectorstore: 1356


## [MY CODE] embedding 변경

텍스트를 위한 embedding 대신 코드에 더 적합한 embedding으로 변경

In [10]:
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings

# CodeBERT embedding 모델 로드
model_name = "microsoft/codebert-base"
embeddings = HuggingFaceEmbeddings(model_name=model_name)



vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="./new_chroma_db",
    collection_name="code_segments"
)

# 데이터 업로드
def add_data_to_chroma(segments, metadata):
    # LangChain의 Document 형식으로 변환
    documents = [
        Document(
            page_content=segment,
            metadata=meta
        ) for segment, meta in zip(segments, metadata)
    ]

    # vectorstore에 documents 추가
    vectorstore.add_documents(documents)

for file_path, code in code_files.items():
    # Split the code into segments (you might want to use a more sophisticated approach)
    segments = code.split('\n')

    # Create metadata for each segment
    metadata = [{'file_path': file_path, 'line_number': i} for i in range(len(segments))]

    # Add the segments and metadata to the vectorstore
    add_data_to_chroma(segments, metadata)

In [12]:
# 쿼리를 통해 관련 벡터 검색
def search_relevant_segments(query: str, top_k: int = 50):
    # similarity_search 메서드를 사용하여 관련 문서 검색
    results = vectorstore.similarity_search(
        query=query,
        k=top_k
    )
    # 검색된 문서의 내용 반환
    return [doc.page_content for doc in results]

# 결과 출력
query = "잠재적인 버그, 스타일 문제, 성능 문제, 보안 취약점 등 특정 코드 블록이나 함수에서 개선할 수 있는 부분"
relevant_segments = search_relevant_segments(query)
relevant_segments

['    @extend_schema(summary="새 토픽 생성")',
 '#     "http://lion-lb-staging-18975820-e3aa639c372a.kr.lb.naverncp.com",',
 '"""Django\'s command-line utility for administrative tasks."""',
 'It exposes the ASGI callable as a module-level variable named ``application``.',
 'os.environ.setdefault("DJANGO_SETTINGS_MODULE", "lion_app.settings")',
 '    os.environ.setdefault("DJANGO_SETTINGS_MODULE", "lion_app.settings.local")',
 'os.environ.setdefault("DJANGO_SETTINGS_MODULE", "lion_app.settings.local")',
 '# Generated by Django 4.2.3 on 2023-08-30 01:29',
 'It exposes the WSGI callable as a module-level variable named ``application``.',
 '# Generated by Django 4.2.3 on 2023-08-30 01:31',
 '# Generated by Django 4.2.3 on 2023-08-04 02:14',
 '    "DEFAULT_PERMISSION_CLASSES": ["rest_framework.permissions.IsAuthenticated"],',
 '# Generated by Django 4.2.3 on 2023-08-16 06:25',
 '    path("api/token/", TokenObtainPairView.as_view(), name="token_obtain_pair"),',
 '    "lion-lb-18904307-df5b6f497f

## [LOG] 코드 블럭이 아닌 라인 형태의 출력

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def generate_code_review(relevant_segments):
    # 프롬프트 템플릿 생성
    prompt = ChatPromptTemplate.from_template("""
    You are a code reviewer. Here are some code segments:
    {segments}

    Please provide:
    1. What was done well.
    2. What can be improved.
    """)

    # LLM 모델 초기화
    llm = ChatOpenAI(model="gpt-4")

    # 체인 실행
    chain = prompt | llm

    # 리뷰 생성
    response = chain.invoke({
        "segments": "\n".join(relevant_segments)
    })

    return response.content

# 리뷰 생성 및 출력
review = generate_code_review(relevant_segments)
print(review)

1. What was done well:

    - The code follows the DRY principle. For example, instead of repeatedly setting the default setting for the Django settings module, it is set once using `os.environ.setdefault("DJANGO_SETTINGS_MODULE", "lion_app.settings")`.
    - The use of descriptive paths (such as "api/schema/" and "api/token/") is commendable. These make the routes more intuitive to users and developers.
    - Good use of comments to provide information regarding what different sections of code do.
    - The use of JWT (JSON Web Tokens) for authentication in the "DEFAULT_AUTHENTICATION_CLASSES" section is a good practice for securing API endpoints. 

2. What can be improved:
   
    - The code seems unstructured and somewhat scattered. The imports, class definitions, constants, and instantiations should each be grouped together for better readability.
    - Redundancy in code: There are many repeated lines such as `"http://lion-lb-18904307-df5b6f497fc4.kr.lb.naverncp.com"`, these could

## [LOG] 향상된 코드리뷰

segement가 비록 코드블록이 아닌 한 줄의 형태로 되어있긴 하지만, 코드가 들어가니 의미있는 답변이 나옴

- 코드의 출처가 나와야 직접 가서 전체 코드를 확인할 수 있을듯

## [FEEDBACK] 코드 블록을 segment로

어떻게 하면, 한 줄이 아니라 코드 블록 단위로 segment를 만들 수 있을까요?